In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# reporting the tuning loss
# our method data
# test_only = True
test_only_true_view_05 = [(1.022, 0.06691), (0.9369, 0.06187), (0.8518, 0.05084), (0.6197, 0.0232)]
# this downward trend does not hold for the training data
# there is something fishy here. Maybe I'm averaging over all frames but only summing over those that don't disappear